In [264]:
import pandas as pd
import numpy as np
import import_ipynb
import sys
import math 
import seaborn as sea
import datetime as dt
import Fuzzy_Prediction_Beta as fz
from datetime import timedelta
from operator import itemgetter
from functools import partial
url = "in.csv"

In [265]:
dataset = pd.read_csv(url, error_bad_lines=False)

In [266]:
selected = dataset[['cod_location','date_id','total_cases']]
selected.head()

,cod_location,date_id,total_cases
0,0,20191231,0.0
1,0,20200101,0.0
2,0,20200102,0.0
3,0,20200103,0.0
4,0,20200104,0.0


In [267]:
selected = selected.groupby('cod_location').tail(5)

In [268]:
selected.head()
#Input na rede neural
selected.to_csv('input.csv')

In [269]:
def aver(input):
    medias = input.groupby(['cod_location'])['total_cases'].mean()
    return medias

In [270]:
def diff(input):
    difference = input.groupby('cod_location')['total_cases'].apply(lambda x: x.tail(1).values[0] - x.head(1).values[0])
    return difference

In [271]:
def first(input):
    first = selected.groupby('cod_location')['total_cases'].apply(lambda x: x.head(1).values[0])
    return first

In [272]:
def last(input):
    last = selected.groupby('cod_location')['total_cases'].apply(lambda x: x.tail(1).values[0])
    return last

In [273]:
def fuzzifyErrorsPos(erro):
    return fz.triangulo(erro, [5, 10, 15])

In [274]:
def fuzzifyErrorPos(erro):
    return fz.triangulo(erro, [0, 5, 10])

In [275]:
def fuzzifyErrorZ(erro):
    return fz.triangulo(erro, [-5, 0, 5])

In [276]:
def fuzzifyErrorNeg(erro):
    return fz.triangulo(erro, [-10,-5,0])

In [277]:
def fuzzifyErrorsNeg(erro):
    return fz.triangulo(erro, [-15,-10,-5])

In [278]:
def fuzzifyErrorDotsPos(erroD):
    return fz.trapezio(erroD,[5,5.5,8.5,9])

In [279]:
def fuzzifyErrorDotPos(erroD):
    return fz.triangulo(erroD,[2,4,6])

In [280]:
def fuzzifyErrorDotZ(erroD):
    return fz.triangulo(erroD,[-2,0,2])

In [281]:
def fuzzifyErrorDotNeg(erroD):
    return fz.triangulo(erroD,[-2,-4,-6])

In [282]:
def fuzzifyErrorDotsNeg(erroD):
    return fz.trapezio(erroD,[-5,-5.5,-8.5,-9])

In [283]:
def agrega(regras, sbot, bot, mid, top, stop):
    result = [0] * 200
    for regra in range(len(regras)):
        for i in range(200):
            if regras[regra][0]>0 and i < 95:
                result[i] = min(regras[regra][0], bot[i])
            if regras[regra][1]>0 and i > 90 and i<110:
                result[i] = min(regras[regra][1], mid[i])
            if regras[regra][2]>0 and i > 80 and i<130:
                result[i] = min(regras[regra][2], top[i])
            if regras[regra][3]>0 and i > 125 and i<150:
                result[i] = min(regras[regra][3], bot[i])
            if regras[regra][4]>0 and i > 145:
                result[i] = min(regras[regra][4], mid[i])
    return result

In [284]:
def OutputSugDecr():
    return fz.getTrapPlot(0, 200, [0, 0, 30, 95], "left")

In [285]:
def OutputDecr():
    return fz.getTriPlot(0, 200, [90, 100, 110])

In [286]:
def OutputPlat():
    return fz.getTriPlot(0, 200, [80, 120, 130])

In [287]:
def OutputCres():
    return fz.getTriPlot(0, 200, [125, 140, 150])

In [288]:
def OutputSugCres():
    return fz.getTrapPlot(0, 200, [145, 180, 200, 200], "right")

In [289]:
def verifRegras(erro, erroD):
    regras = [[0] * 25 for i in range(5)]
    
    fuzzifyErrorsNega = fuzzifyErrorsNeg(erro)
    fuzzifyErrorNega = fuzzifyErrorNeg(erro)
    fuzzifyErrorZe = fuzzifyErrorZ(erro)
    fuzzifyErrorPosi= fuzzifyErrorPos(erro)
    fuzzifyErrorsPosi= fuzzifyErrorsPos(erro)
    
    fuzzifyErrorDotsNega = fuzzifyErrorDotsNeg(erroD)
    fuzzifyErrorDotNega = fuzzifyErrorDotNeg(erroD)
    fuzzifyErrorDotZe = fuzzifyErrorDotZ(erroD)
    fuzzifyErrorDotPosi = fuzzifyErrorDotPos(erroD)
    fuzzifyErrorDotsPosi = fuzzifyErrorDotsPos(erroD)
    
    regras[0][0] = min(fuzzifyErrorsNega,fuzzifyErrorDotsNega)
    regras[0][1] = min(fuzzifyErrorNega,fuzzifyErrorDotsNega)
    regras[0][2] = min(fuzzifyErrorZe,fuzzifyErrorDotsNega)
    regras[0][3] = min(fuzzifyErrorPosi,fuzzifyErrorDotsNega)
    regras[0][4] = min(fuzzifyErrorsPosi,fuzzifyErrorDotsNega)
    
    regras[1][0] = min(fuzzifyErrorsNega,fuzzifyErrorDotNega)
    regras[1][1] = min(fuzzifyErrorNega,fuzzifyErrorDotNega)
    regras[1][2] = min(fuzzifyErrorZe,fuzzifyErrorDotNega)
    regras[1][3] = min(fuzzifyErrorPosi,fuzzifyErrorDotNega)
    regras[1][4] = min(fuzzifyErrorsPosi,fuzzifyErrorDotNega)
    
    regras[2][0] = min(fuzzifyErrorsNega,fuzzifyErrorDotZe)
    regras[2][1] = min(fuzzifyErrorNega,fuzzifyErrorDotZe)
    regras[2][2] = min(fuzzifyErrorZe,fuzzifyErrorDotZe)
    regras[2][3] = min(fuzzifyErrorPosi,fuzzifyErrorDotZe)
    regras[2][4] = min(fuzzifyErrorsPosi,fuzzifyErrorDotZe)
    
    regras[3][0] = min(fuzzifyErrorsNega,fuzzifyErrorDotPosi)
    regras[3][1] = min(fuzzifyErrorNega,fuzzifyErrorDotPosi)
    regras[3][2] = min(fuzzifyErrorZe,fuzzifyErrorDotPosi)
    regras[3][3] = min(fuzzifyErrorPosi,fuzzifyErrorDotPosi)
    regras[3][4] = min(fuzzifyErrorsPosi,fuzzifyErrorDotPosi)
    
    regras[4][0] = min(fuzzifyErrorsNega,fuzzifyErrorDotsPosi)
    regras[4][1] = min(fuzzifyErrorNega,fuzzifyErrorDotsPosi)
    regras[4][2] = min(fuzzifyErrorZe,fuzzifyErrorDotsPosi)
    regras[4][3] = min(fuzzifyErrorPosi,fuzzifyErrorDotsPosi)
    regras[4][4] = min(fuzzifyErrorsPosi,fuzzifyErrorDotsPosi)
    
    return regras

In [290]:
media = aver(selected)
diffe = diff(selected)
first = first(selected)
target = media - (media * 28) / 100

In [291]:
prevError = target - first
currError = target - diffe

In [292]:
error = 1.5
#currError
dervError = 2.75
#first - diffe

In [293]:
##
Regras = verifRegras(error, dervError)
vA = agrega(Regras, OutputSugDecr(), OutputDecr(), OutputPlat(), OutputCres(), OutputSugCres())
centroide = fz.getCentroid(vA)

print(error)
print(dervError)
print(centroide)
##

ZeroDivisionError: float division by zero